# LGBM

In [1]:
import numpy as np
import pandas as pd
import os
import random

import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np

## 1. 데이터 로딩

In [2]:
data_dir = '/opt/ml/input/data/' 
csv_file_path = os.path.join(data_dir, 'elo.csv') 
df = pd.read_csv(csv_file_path,  parse_dates=['Timestamp'])

In [3]:
data_dir = '/opt/ml/input/data/' 
csv_file_path = os.path.join(data_dir, 'elo2.csv') 
test = pd.read_csv(csv_file_path,  parse_dates=['Timestamp'])

In [4]:
test

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elo
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,0.752296
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,0.416693
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,0.281076
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,0.382929
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,0.166432
...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,0.132075
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,0.410814
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,0.621880
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,0.760312


# train, test concat

In [5]:
# df = pd.concat([train, test])
def feature_engineering_(df):

    # 유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=["userID", "Timestamp"], inplace=True)

    # 카테고리형 feature
    categories = ["assessmentItemID", "testId"] 

    for category in categories:
        df[category] = df[category].astype("category")

    return df


df = feature_engineering_(df)
test = feature_engineering_(test)

## 2. Feature Engineering

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2526700 entries, 0 to 2526699
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   userID            int64         
 1   assessmentItemID  category      
 2   testId            category      
 3   answerCode        int64         
 4   Timestamp         datetime64[ns]
 5   KnowledgeTag      int64         
 6   dataset           int64         
 7   elo               float64       
dtypes: category(2), datetime64[ns](1), float64(1), int64(4)
memory usage: 144.9 MB


In [7]:
df = df[df.answerCode != -1]

In [8]:
df[df['answerCode'] == -1]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,elo


In [9]:
def feature_engineering(df):
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    # df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer_cumsum'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer_cumcount'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer_cumsum']/df['user_total_answer_cumcount']

    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
    correct_t.columns = ["test_mean",  'test_sum']
    
    df['Timestamp'] = df['Timestamp'].astype(str)
    df['month'] = df['Timestamp'].str[5:7].astype(int)
    df['category_2'] = df['assessmentItemID'].str[2].astype(int)
    df['test_paper'] = df['assessmentItemID'].str[4:7].astype(int)
    df['problem_id'] = df['assessmentItemID'].str[-3:].astype(int)

    df = pd.merge(df, correct_t, on=['testId'], how="left")
    # df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    # 결측치 처리
    df = df.fillna(0)
    
    df['lagtime'] = pd.to_datetime(df['Timestamp'],infer_datetime_format=True)
    df['lagtime'] = df['lagtime'].apply(lambda x: x.timestamp())
    diff = df.loc[:,['userID','lagtime']].groupby('userID').diff(periods=-1)
    df['lagtime'] = diff['lagtime'].apply(abs)
    df.loc[df['lagtime']>30000, 'lagtime'] = 30000

    # 해당 문제 푸는데 걸린 시간
    df['lagtime'].fillna(0, inplace=True)
    df.lagtime=df.lagtime.astype('int32')
    
    lagtime_agg = df.groupby('userID')['lagtime'].agg(['mean'])
    df['lagtime_mean'] = df['userID'].map(lagtime_agg['mean'])
    df.lagtime_mean=df.lagtime_mean.astype('int32')
    
    # 시도 횟수
    df["attempt_no"] = 1
    df.attempt_no=df.attempt_no.astype('int8')
    df["attempt_no"] = df[["userID","assessmentItemID",'attempt_no']].groupby(["userID","assessmentItemID"])["attempt_no"].cumsum()

    user_agg = df.groupby('userID')['answerCode'].agg(['sum', 'count'])
    assessment_agg = df.groupby('assessmentItemID')['answerCode'].agg(['sum', 'count','var'])
    test_agg = df.groupby('testId')['answerCode'].agg(['sum', 'count','var'])
    Tag_agg = df.groupby('KnowledgeTag')['answerCode'].agg(['sum', 'count','var'])
    attempt_no_agg=df.groupby(["userID","assessmentItemID"])["attempt_no"].agg(['sum'])

    user_agg=user_agg.astype('int16')
    assessment_agg = assessment_agg.astype('float32')
    test_agg = test_agg.astype('float32')
    Tag_agg= Tag_agg.astype('float32')
    attempt_no_agg = attempt_no_agg.astype('int8')
    
    df['assessmentItemId_cnt'] = df['assessmentItemID'].map(assessment_agg['count']).astype('int32')
    df['assessmentItemId_sum'] = df['assessmentItemID'].map(assessment_agg['sum']).astype('int32')
    df['assessmentItemId_mean'] = df['assessmentItemID'].map(assessment_agg['sum'] / assessment_agg['count'])
    df.assessmentItemId_mean=df.assessmentItemId_mean.astype('float16')

    df['KnowledgeTag_sum'] = df['KnowledgeTag'].map(Tag_agg['sum']).astype('int32')
    df['KnowledgeTag_std'] = df['KnowledgeTag'].map(Tag_agg['var']).astype('float16')
    df['KnowledgeTag_mean'] = df['KnowledgeTag'].map(Tag_agg['sum'] / Tag_agg['count'])
    df.KnowledgeTag_mean=df.KnowledgeTag_mean.astype('float16')
    
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['hour'] = df['Timestamp'].dt.hour
    time_agg = df.groupby('hour')['answerCode'].agg(['mean', 'sum'])
    time_agg['sum'] = time_agg['sum'] / time_agg['sum'].max()

    df['hour_mean'] = df['hour'].map(time_agg['mean']).astype('float16')
    df['hour_sum'] = df['hour'].map(time_agg['sum']).astype('float16')
    
    return df


In [10]:
df = feature_engineering(df)

In [11]:
test = feature_engineering(test)

In [12]:
import gc
gc.get_count()
gc.collect()
gc.get_count()

(55, 0, 0)

In [13]:
test

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elo,user_correct_answer_cumsum,user_total_answer_cumcount,user_acc,...,attempt_no,assessmentItemId_cnt,assessmentItemId_sum,assessmentItemId_mean,KnowledgeTag_sum,KnowledgeTag_std,KnowledgeTag_mean,hour,hour_mean,hour_sum
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,0.752296,0.0,0,0.000000,...,1,32,24,0.750000,91,0.244019,0.586914,10,0.701172,0.965332
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,0.416693,1.0,1,1.000000,...,1,32,18,0.562500,91,0.244019,0.586914,10,0.701172,0.965332
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,0.281076,2.0,2,1.000000,...,1,32,11,0.343750,123,0.243286,0.588379,10,0.701172,0.965332
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,0.382929,2.0,3,0.666667,...,1,32,16,0.500000,123,0.243286,0.588379,10,0.701172,0.965332
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,0.166432,2.0,4,0.500000,...,1,32,9,0.281250,252,0.243896,0.582031,10,0.701172,0.965332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,0.132075,8.0,11,0.727273,...,1,42,16,0.380859,193,0.249634,0.584961,23,0.656738,0.471191
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,0.410814,8.0,12,0.666667,...,1,42,22,0.523926,193,0.249634,0.584961,23,0.656738,0.471191
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,0.621880,9.0,13,0.692308,...,1,42,32,0.761719,111,0.226807,0.725586,23,0.656738,0.471191
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,0.760312,10.0,14,0.714286,...,1,42,36,0.856934,111,0.226807,0.725586,23,0.656738,0.471191


## 3. Train/Test 데이터 셋 분리

- 아래와 같이 split을 진행함
- custom split은 필수적임, 맨마지막 interaction을 가져와야 되니까
- 반복문을 이용해서 train 시키고, 나눠주면 될 듯

In [14]:
# Inference 위해 미리 나누어 놓음
# train = df[df.dataset == 1]
# test = test = df[df.dataset == 2]

In [15]:
def all_train_valid_split(df):
    train = df[df.dataset == 1]

    test = df[(df.dataset == 2) & (df.answerCode != -1)]  # -1 인 answerCode 제외
    
    test = test[test["userID"] != test["userID"].shift(-1)]

    return train, test

In [16]:
# train과 valid 데이터셋은 사용자 별로 묶어서 분리를 해주어야함
random.seed(42)
def custom_train_valid_split(df, ratio=0.9, split=True):
    
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)


    train = df[df['userID'].isin(user_ids)]
    valid = df[df['userID'].isin(user_ids) == False]

    #valid데이터셋은 각 유저의 마지막 interaction만 추출
    valid = valid[valid['userID'] != valid['userID'].shift(-1)]
    
    return train, valid

In [17]:
# 유저별 분리
train, valid = all_train_valid_split(df)

# 사용할 Feature 설정
FEATS = ['KnowledgeTag',
 'KnowledgeTag_mean',
 'KnowledgeTag_std',
 'KnowledgeTag_sum',
#  'Timestamp',
#  'assessmentItemID',
 'assessmentItemId_cnt',
 'assessmentItemId_mean',
 'assessmentItemId_sum',
 'attempt_no',
 'category_2',
#  'dataset',
 'elo',
 'hour',
 'hour_mean',
 'hour_sum',
 'lagtime',
 'lagtime_mean',
 'month',
 'problem_id',
 'testId',
 'test_mean',
 'test_paper',
 'test_sum',
#  'userID',
 'user_acc',
 'user_correct_answer_cumsum',
 'user_total_answer_cumcount']


y_train = train['answerCode']
train = train.drop(['answerCode'], axis=1)

y_valid = valid['answerCode']
valid = valid.drop(['answerCode'], axis=1)

In [18]:
# dataset에 포함된 feature 목록
sorted(list(train.columns))

['KnowledgeTag',
 'KnowledgeTag_mean',
 'KnowledgeTag_std',
 'KnowledgeTag_sum',
 'Timestamp',
 'assessmentItemID',
 'assessmentItemId_cnt',
 'assessmentItemId_mean',
 'assessmentItemId_sum',
 'attempt_no',
 'category_2',
 'dataset',
 'elo',
 'hour',
 'hour_mean',
 'hour_sum',
 'lagtime',
 'lagtime_mean',
 'month',
 'problem_id',
 'testId',
 'test_mean',
 'test_paper',
 'test_sum',
 'userID',
 'user_acc',
 'user_correct_answer_cumsum',
 'user_total_answer_cumcount']

In [19]:
lgb_train = lgb.Dataset(train[FEATS], y_train)
lgb_test = lgb.Dataset(valid[FEATS], y_valid)

## 4. 훈련 및 검증

In [20]:
# 'num_leaves': 613, 'max_depth': 13, 'learning_rate': 0.013406787694932351, 'n_estimators': 200, 'min_child_samples': 57, 'bagging_fraction': 0.5329052554587342

In [21]:
model = lgb.train(
    {'objective': 'binary',
    'max_depth':13,
    'num_leaves': 613,
    'n_estimators':200,
    "boosting": "gbdt",
    'num_leaves':80,
    'min_child_samples':57                    ,
    "seed": 42,
    'learning_rate': 0.013406787694932351,
    "bagging_fraction": 0.5329052554587342,
    'metric' : 'auc'}, 
    train_set = lgb_train,
    valid_sets=[lgb_train, lgb_test],
    verbose_eval=100,
    num_boost_round=1000
)

preds = model.predict(valid[FEATS])
acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_valid, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1483205, number of negative: 783381
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.362967 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5381
[LightGBM] [Info] Number of data points in the train set: 2266586, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654378 -> initscore=0.638341
[LightGBM] [Info] Start training from score 0.638341


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	training's auc: 0.828547	valid_1's auc: 0.806489
[200]	training's auc: 0.834149	valid_1's auc: 0.80089
VALID AUC : 0.8008895196155167 ACC : 0.7338709677419355



In [21]:
# df_importances = pd.DataFrame(FEATS, columns=['feature_name'])
# df_importances['gain'] = model.feature_importance('gain').astype(int)
# df_importances = df_importances.sort_values('gain', ascending=False).reset_index(drop=True)
# df_importances.head(50)

# CV - Optuna

## seed 고정 필요

In [22]:
np.random.seed(42)
user_idx = np.arange(7441)
np.random.shuffle(user_idx)
user_idx

array([5829, 1760, 5498, ..., 5390,  860, 7270])

In [23]:
folds = []
for i in range(1, 11):
    start = (7441// 10) * (i-1)
    end = (7441// 10) * i
    
    valid_idx = user_idx[start:end]
    train_idx = list(user_idx[:start]) + list(user_idx[end:])
    
    folds.append((valid_idx, train_idx))
    

In [24]:
# folds[0] = list(folds[0])
# folds[1] = list(folds[1])
# folds[0][0] = list(folds[0][0]) + list(folds[1][0])

In [29]:
for i in range(10):
    print(len(folds[i][1]))

6697
6697
6697
6697
6697
6697
6697
6697
6697
6697


In [30]:
df.loc[df['userID'].isin(folds[0][1]), :]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,elo,user_correct_answer_cumsum,user_total_answer_cumcount,...,attempt_no,assessmentItemId_cnt,assessmentItemId_sum,assessmentItemId_mean,KnowledgeTag_sum,KnowledgeTag_std,KnowledgeTag_mean,hour,hour_mean,hour_sum
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,0.980768,0.0,0,...,1,250,246,0.983887,718,0.040894,0.957520,0,0.650879,0.617676
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,0.973315,1.0,1,...,1,250,242,0.967773,3439,0.076050,0.916992,0,0.650879,0.617676
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,0.947292,2.0,2,...,1,250,229,0.916016,3439,0.076050,0.916992,0,0.650879,0.617676
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,0.974914,3.0,3,...,1,250,243,0.972168,3439,0.076050,0.916992,0,0.650879,0.617676
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,0.961391,4.0,4,...,1,250,237,0.948242,3439,0.076050,0.916992,0,0.650879,0.617676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525942,7440,A030197001,A030000197,1,2020-10-21 08:31:45,1984,1,0.717324,5.0,10,...,1,300,264,0.879883,2192,0.190430,0.744141,8,0.669922,0.993164
2525943,7440,A030197002,A030000197,0,2020-10-21 08:32:46,1984,1,0.181081,6.0,11,...,1,300,141,0.469971,2192,0.190430,0.744141,8,0.669922,0.993164
2525944,7440,A030197003,A030000197,0,2020-10-21 08:33:15,1984,1,0.719288,6.0,12,...,1,300,268,0.893555,2192,0.190430,0.744141,8,0.669922,0.993164
2525945,7440,A030197004,A030000197,0,2020-10-21 08:33:17,1984,1,0.570404,6.0,13,...,1,300,251,0.836426,2192,0.190430,0.744141,8,0.669922,0.993164


In [31]:
test_df = df[df.dataset == 2]
# test_df = test
# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df["userID"] != test_df["userID"].shift(-1)]

# DROP ANSWERCODE
test_df = test_df.drop(["answerCode"], axis=1)

# MAKE PREDICTION
# total_preds = model.predict(test_df[FEATS])

In [32]:
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import SuccessiveHalvingPruner

def objective(trial):
    # lgb_train = lgb.Dataset(train[FEATS], y_train)
    # lgb_test = lgb.Dataset(valid[FEATS], y_valid)

    param = {
        'objective': 'binary', # 회귀
        'verbose': -1,
        'metric': 'auc', 
        "boosting_type": "gbdt",
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024), 
        'max_depth': trial.suggest_int('max_depth', 1, 14), 
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1), 
        'n_estimators': trial.suggest_int('n_estimators', 5, 1000), 
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 1050), 
        'bagging_fraction': trial.suggest_float('bagging_fraction',0.38, 0.85),
        # 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        # 'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'seed': 42
    }
    
    model = lgb.train(param, train_set = lgb.Dataset(train, y_train),
                        valid_sets=[lgb.Dataset(train, y_train), lgb.Dataset(valid, y_valid)])
    preds = model.predict(valid)
    
    acc = accuracy_score(y_valid, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_valid, preds)

    return auc
    
for fold in range(0,10):
    print(f'===================================={fold+1}============================================')
        
    valid_idx, train_idx = folds[fold]
    
    train = df.loc[df['userID'].isin(train_idx), :]
    y_train = train['answerCode']
    train = train[FEATS]
    
    valid = df.loc[df['userID'].isin(valid_idx), :]
    valid = valid[valid["userID"] != valid["userID"].shift(-1)]
    y_valid = valid['answerCode']
    valid = valid[FEATS]
    

    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(
        study_name = 'lgbm_parameter_opt',
        direction = 'maximize',
        sampler = sampler,
        )
    
    study.optimize(objective, n_trials=15)

    model = lgb.LGBMRegressor(**study.best_params)
    model.fit(train, y_train,
                eval_set=(valid,y_valid),
                eval_metric='auc', verbose=False, early_stopping_rounds=100)
    
    pred = model.predict(test_df[FEATS])
    test_df[f'pred_{fold}'] = pred
    print(f'================================================================================\n\n')

====================================1============================================


[I 2022-12-02 20:03:20,297] A new study created in memory with name: lgbm_parameter_opt
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden

[1]	valid_0's auc: 0.827318	valid_1's auc: 0.787331
[2]	valid_0's auc: 0.829953	valid_1's auc: 0.795375
[3]	valid_0's auc: 0.831787	valid_1's auc: 0.796075
[4]	valid_0's auc: 0.833011	valid_1's auc: 0.803389
[5]	valid_0's auc: 0.833885	valid_1's auc: 0.803145
[6]	valid_0's auc: 0.834713	valid_1's auc: 0.805581
[7]	valid_0's auc: 0.835412	valid_1's auc: 0.807526
[8]	valid_0's auc: 0.836129	valid_1's auc: 0.810433
[9]	valid_0's auc: 0.836742	valid_1's auc: 0.810546
[10]	valid_0's auc: 0.837355	valid_1's auc: 0.810807
[11]	valid_0's auc: 0.83795	valid_1's auc: 0.811207
[12]	valid_0's auc: 0.838462	valid_1's auc: 0.811123
[13]	valid_0's auc: 0.838995	valid_1's auc: 0.811471
[14]	valid_0's auc: 0.839513	valid_1's auc: 0.810891
[15]	valid_0's auc: 0.839976	valid_1's auc: 0.812089
[16]	valid_0's auc: 0.840472	valid_1's auc: 0.812433
[17]	valid_0's auc: 0.840882	valid_1's auc: 0.811954
[18]	valid_0's auc: 0.841359	valid_1's auc: 0.811486
[19]	valid_0's auc: 0.841821	valid_1's auc: 0.811591
[20

[I 2022-12-02 20:06:31,523] Trial 0 finished with value: 0.80355548457814 and parameters: {'num_leaves': 385, 'max_depth': 14, 'learning_rate': 0.07453942447208349, 'n_estimators': 601, 'min_child_samples': 172, 'bagging_fraction': 0.45331742455801527}. Best is trial 0 with value: 0.80355548457814.
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters fr

[1]	valid_0's auc: 0.815034	valid_1's auc: 0.780011
[2]	valid_0's auc: 0.81735	valid_1's auc: 0.793219
[3]	valid_0's auc: 0.819276	valid_1's auc: 0.794924
[4]	valid_0's auc: 0.820044	valid_1's auc: 0.790747
[5]	valid_0's auc: 0.820593	valid_1's auc: 0.790859
[6]	valid_0's auc: 0.821185	valid_1's auc: 0.792122
[7]	valid_0's auc: 0.821754	valid_1's auc: 0.794765
[8]	valid_0's auc: 0.82233	valid_1's auc: 0.796511
[9]	valid_0's auc: 0.822705	valid_1's auc: 0.797407
[10]	valid_0's auc: 0.822999	valid_1's auc: 0.797788
[11]	valid_0's auc: 0.823331	valid_1's auc: 0.796623
[12]	valid_0's auc: 0.823721	valid_1's auc: 0.797734
[13]	valid_0's auc: 0.824056	valid_1's auc: 0.796227
[14]	valid_0's auc: 0.824321	valid_1's auc: 0.794122
[15]	valid_0's auc: 0.824581	valid_1's auc: 0.794543
[16]	valid_0's auc: 0.82489	valid_1's auc: 0.797708
[17]	valid_0's auc: 0.825153	valid_1's auc: 0.79589
[18]	valid_0's auc: 0.825485	valid_1's auc: 0.797922
[19]	valid_0's auc: 0.82582	valid_1's auc: 0.799944
[20]	va

...

[I 2022-12-02 21:28:58,745] Trial 14 finished with value: 0.7882880277617119 and parameters: {'num_leaves': 719, 'max_depth': 6, 'learning_rate': 0.03660769373729209, 'n_estimators': 231, 'min_child_samples': 492, 'bagging_fraction': 0.3830033033330727}. Best is trial 9 with value: 0.7994975419317525.


[231]	valid_0's auc: 0.836884	valid_1's auc: 0.788288


/opt/conda/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] bagging_fraction is set=0.5329052554587342, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5329052554587342


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


# 5. Inference

In [ ]:
# test_df['rating'] = (test_df['pred_0'] + test_df['pred_1'] + test_df['pred_2'] + test_df['pred_3'] + test_df['pred_4'] +test_df['pred_5'] +
#                                             test_df['pred_6'] + test_df['pred_7'] + test_df['pred_8'] + test_df['pred_9']) / 10
# total_preds = (test_df['pred_0'] + test_df['pred_1'] + test_df['pred_2'] + test_df['pred_3'] + test_df['pred_4'] +test_df['pred_5'] +
                                            # test_df['pred_6'] + test_df['pred_7'] + test_df['pred_8'] + test_df['pred_9'] ) / 10
# test = test_df[['userID', 'rating']]
# test.to_csv('../submit/CAT_10Fold.csv', index = False)
# test.to_csv('submit/LGBM.csv', index = False)

In [28]:
test_df = test

# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df["userID"] != test_df["userID"].shift(-1)]

# DROP ANSWERCODE
test_df = test_df.drop(["answerCode"], axis=1)

# MAKE PREDICTION
total_preds = model.predict(test_df[FEATS])

In [26]:
# test_df[FEATS]

In [27]:
# SAVE OUTPUT
output_dir = 'output/'
write_path = os.path.join(output_dir, "LJH_LGBM_submission_best_parameters.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write('{},{}\n'.format(id,p))

writing prediction : output/LJH_LGBM_submission_best_parameters.csv


###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.

